Import libraries

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn.linear_model as sklm
import sklearn.pipeline
import sklearn.model_selection as skms
import sklearn.feature_selection 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform

# Import our filess
from load_train_data import load_data 
from clean_data import tokenize_text, create_word_list, create_dict, make_feature_vector

In [2]:
website_list, review_list, rating_list = load_data('x_train.csv', 'y_train.csv')
tokens = tokenize_text(review_list) #A list of 28459 cleaned words
# print(len(tokens))

word_count_dict = create_word_list(tokens) # dictionary of 4883 unique word counts
sorted_tokens = list(sorted(word_count_dict, key=word_count_dict.get, reverse=True))
vocab_dict = create_dict(sorted_tokens)

# clean up the data INSTEAD of tokens = tokenize_text(review_list)
vectorizer = CountVectorizer(stop_words = 'english', ngram_range=(1,1), min_df=3, max_df=0.08, binary=False, vocabulary=vocab_dict) #filter out words over 0.08 occurence
# vectorizer = CountVectorizer(analyzer = 'word',tokenizer=lambda txt: txt.split(),token_pattern = str,ngram_range=(1,1), min_df=0.0, max_df=1.0, binary=False)

X = vectorizer.fit(review_list, rating_list) 

# Save output to text file
with open('wordFreq.txt', 'w') as f:
    f.write(str(word_count_dict))

with open('afterCountVectorizer.txt', 'w') as f:
    # f.write(str(word_count_dict))
    f.write(str(vectorizer.get_feature_names_out().tolist()))


# with open('output.txt', 'r') as f:
#     print(f.read())

#print(vectorizer.vocabulary_)


In [3]:
#print(vectorizer.get_feature_names_out()[703])
#print(review_list)
# print(vectorizer.get_feature_names_out()[0:100])

sorted_tokens = list(sorted(word_count_dict, key=word_count_dict.get, reverse=True))

print(type(sorted_tokens)) # list of tokens sorted by frequency

# Print all words in list and their frequencies from the dictionary
for w in sorted_tokens:
    print("%5d %s" % (word_count_dict[w], w))
    
# Create a dictionary of the sorted tokens list
vocab_dict = create_dict(sorted_tokens)
print(sorted_tokens)



In [4]:
# Turning reviews into feature vectors
N = len(review_list)
V = len(sorted_tokens)

x_tr_NV = np.zeros((N,V))   # N x V matrix of feature vectors


for nn, review_line in enumerate(review_list[0:5]):
    x_tr_NV[nn] = make_feature_vector(review_line, vocab_dict)
    print(review_line)
    print(sum(x_tr_NV[nn]))
    # print(review_list[nn])
# TODO are we using the right inputs and functions here?
# with open('x_tr_feature.txt', 'w') as f:
#     # f.write(str(word_count_dict))
#     f.write(str(x_tr_NV[5].tolist()))

# print(np.sum(x_tr_NV[5]))
# print(review_list[5])

Oh and I forgot to also mention the weird color effect it has on your phone.
Oh and I forgot to also mention the weird color effect it has on your phone.
16.0
THAT one didn't work either.
THAT one didn't work either.
5.0
Waste of 13 bucks.
Waste of 13 bucks.
4.0
Product is useless, since it does not have enough charging current to charge the 2 cellphones I was planning to use it with.
Product is useless, since it does not have enough charging current to charge the 2 cellphones I was planning to use it with.
23.0
None of the three sizes they sent with the headset would stay in my ears.
None of the three sizes they sent with the headset would stay in my ears.
15.0


In [13]:
my_bow_classifier_pipeline = sklearn.pipeline.Pipeline([
    ('my_bow_feature_extractor', CountVectorizer(min_df=1, max_df=1.0, ngram_range=(1,1))),
    ('my_classifier', sklm.LogisticRegression(C=1.0, max_iter=200, random_state=101)),
])

my_bow_classifier_pipeline.fit(review_list, rating_list)
my_bow_classifier_pipeline.predict(review_list)
my_bow_classifier_pipeline.score(review_list, rating_list)
probs = my_bow_classifier_pipeline.predict_proba(review_list)
acc1 = roc_auc_score(rating_list, probs[:,1])
print("Training accuracy: %.3f" % acc1)


Training accuracy: 0.994


# sklearn.model_selection.RandomizedSearchCV
Look for if there is an industry standard for what % of your training data should be the number of iterations and just use that and explain that's what you considered instead of evaluating that you have reached an optimum minima.

In [5]:
# Define a random search cross validation to generate optimum C, penalty and max_iter for a logistic regression classifier
logistic = sklm.LogisticRegression(solver='liblinear', max_iter=1000)
distributions = dict(C=np.logspace(-9,-6,31), penalty = ['l2', 'l1'])
randClassifier = skms.RandomizedSearchCV(logistic, distributions, n_iter=100, cv=10, verbose=0, random_state=0, error_score='raise', return_train_score=True)

from sklearn.utils import shuffle

# Tried to add shuffle before passing it into the randomizedSearchCV to see if it would improve the fit. It basically didn't - tiny improvement.
# x_tr_NV, rating_list = shuffle(x_tr_NV, rating_list, random_state=0)
#print(type(randClassifier))
#randClassifier.fit(x_tr_NV, rating_list)
# TODO ensure we're passing the right x and y variables and that they match each other (right ratings with each review) into the classifier.

#print(type(randClassifier))

In [6]:
newregress = sklm.LogisticRegression()
newregress.fit(x_tr_NV, rating_list)
yhat_tr_N = newregress.predict(x_tr_NV)
acc_tr = np.mean(yhat_tr_N == rating_list)

print("Training accuracy: %.3f" % acc_tr)


Training accuracy: 0.502


In [7]:
# Just experimenting to ensure that if I pass 2 arrays in that shuffle shuffles both in the same way
# a = np.arange(18)
# b = np.arange(18)+7

# a, b = shuffle(a, b, random_state=0)

# print(a)
# print(b-7)

In [8]:
# Probability of 0 and 1 [2400,2]
yhat = randClassifier.predict(x_tr_NV)
print((yhat.shape))

# Save yhat to text file
with open('yhat.txt', 'w') as f:
    f.write(str(yhat.tolist()))

acccuracy = np.mean(yhat == rating_list)
print("Training accuracy = %f" % acccuracy)

NameError: name 'randClassifier' is not defined

In [ ]:
# Evaluate the AUROC accuracy of the classifier on the training data
acc = roc_auc_score(rating_list, yhat[:,])
print("Training accuracy: %.3f" % acc)

Training accuracy: 0.502


In [ ]:
# make_feature_vector()

In [ ]:
# from sklearn.datasets import load_iris
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform
# iris = load_iris()
# logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200, random_state=0)
# distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])
# clf = RandomizedSearchCV(logistic, distributions, random_state=0)
# search = clf.fit(iris.data, iris.target)
# search.best_params_